In [71]:
%reset -f
import requests
import json
import re
from bs4 import BeautifulSoup

home_url = "https://animepahe.ru/"
anime_url = home_url+"anime/"
api_url_extension = "api?m="
search_url_extension = api_url_extension+"search&q="
quality = "720p"
sub_or_dub = "sub"

#keyword = input("Enter the name of the anime you want to download> ")
keyword = "Boku no hero"

#Searches for the anime in the animepahe database
def Searcher(keyword):
    full_search_url = home_url+search_url_extension+keyword
    response = requests.get(full_search_url)
    results = json.loads(response.content.decode("UTF-8"))["data"]
    return results

results = Searcher(keyword)

In [72]:
#Prompts the user to select the index of the anime they want from a list of the search results and returns the id of the chosen anime
def AnimeSelection(results):
    while "anime_id" not in locals():    
        print("Please enter the number belonging to the anime you want from the list below")
        for index, result in enumerate(results):
            print(index+1, result["title"])

        index_of_chosen_anime = int(input("Number> "))-1

        if not index_of_chosen_anime <= 0 and not len(results) < index_of_chosen_anime:
            for index, result in enumerate(results):
                if index == index_of_chosen_anime:
                    anime_id = result["session"]
                    return anime_id
        else:
            print("\nInvalid number Senpwai")

anime_id = AnimeSelection(results)



Please enter the number belonging to the anime you want from the list below
1 Boku no Hero Academia
2 Boku no Hero Academia 2nd Season
3 Boku no Hero Academia 3rd Season
4 Boku no Hero Academia 4th Season
5 Boku no Hero Academia 5th Season
6 Boku no Hero Academia 6th Season
7 Boku no Hero Academia the Movie 1: Futari no Hero
8 Boku no Hero Academia the Movie 2: Heroes:Rising

Invalid number Senpwai
Please enter the number belonging to the anime you want from the list below
1 Boku no Hero Academia
2 Boku no Hero Academia 2nd Season
3 Boku no Hero Academia 3rd Season
4 Boku no Hero Academia 4th Season
5 Boku no Hero Academia 5th Season
6 Boku no Hero Academia 6th Season
7 Boku no Hero Academia the Movie 1: Futari no Hero
8 Boku no Hero Academia the Movie 2: Heroes:Rising

Invalid number Senpwai
Please enter the number belonging to the anime you want from the list below
1 Boku no Hero Academia
2 Boku no Hero Academia 2nd Season
3 Boku no Hero Academia 3rd Season
4 Boku no Hero Academia 4t

In [73]:

#Issues a GET command to the server together with the id of the anime and returns a list of the links(not donwload links) to all the episodes of that anime
def EpisodeLinks(anime_id):
    response = requests.get(home_url+api_url_extension+"release&id="+anime_id+"&sort=episode_asc")
    episode_data = json.loads(response.content.decode("UTF-8"))["data"]
    episode_sessions = [episode["session"] for episode in episode_data]
    episode_links = [home_url+"play/"+anime_id+"/"+str(episode_session) for episode_session in episode_sessions]
    return episode_links

episode_links = EpisodeLinks(anime_id)

In [74]:
def DownloadData(episode_links):
    download_data = []
    for episode_link in episode_links:
        episode_page = requests.get(episode_link).content
        soup = BeautifulSoup(episode_page, "html.parser")
        download_data.append(soup.find_all("a", class_="dropdown-item", target="_blank"))
    #Scrapes the download data of each episode and stores the links for each quality and dub or sub in a list which is contained in another list containing all episodes
    download_links = [[download_link["href"] for download_link in episode_data] for episode_data in download_data]
    #Scrapes the download data of each episode and stores the info for each quality and dub or sub in a list which is contained in another list containing all episodes
    download_info = [[episode_info.text.strip() for episode_info in episode_data] for episode_data in download_data] 
    return (download_links, download_info)   
download_links, download_info = DownloadData(episode_links)

In [89]:
#Returns the sizes of the downloads
def DownloadSizes(download_info):
    #You dont have to understand what happens here cause even I don't lol, ChatGpt for the win
    #The function uses a regular expression to find an expression that matches it's "syntax" starts with ( and ends with MB)
    #This match is then striped of the MB part in order to convert it into an integer, but before that it is converted into a string cause re.findall returns a string
    #The string is then converted into an integer and voila we have the sizes of our episodes
    download_sizes = [[int("".join(re.findall(r'\((.*?)MB\)'.strip("MB"), episode_info))) for episode_info in episode_data] for episode_data in download_info]
    return download_sizes

download_sizes = DownloadSizes(download_info)
download_sizes
    


[[55, 121, 190, 52, 116, 182],
 [69, 155, 242, 67, 150, 235],
 [78, 183, 276, 76, 179, 269],
 [72, 164, 244, 70, 159, 235],
 [76, 172, 272, 73, 168, 265],
 [54, 117, 182, 52, 113, 175],
 [56, 141, 244, 54, 138, 239],
 [64, 145, 225, 62, 141, 218],
 [56, 122, 192, 54, 119, 187],
 [67, 153, 255, 65, 150, 250],
 [56, 125, 196, 55, 122, 191],
 [58, 127, 199, 56, 124, 193],
 [46, 98, 150, 44, 94, 144],
 [54, 121, 188, 52, 117, 181],
 [52, 115, 179, 50, 112, 174],
 [54, 124, 197, 52, 121, 192],
 [58, 134, 218, 57, 132, 214],
 [46, 100, 157, 44, 96, 150],
 [49, 106, 163, 48, 103, 157],
 [50, 110, 169, 48, 105, 162],
 [53, 113, 172, 51, 109, 166],
 [68, 159, 249, 66, 157, 242],
 [67, 155, 243, 65, 152, 237],
 [68, 161, 241, 66, 158, 234],
 [43, 92, 142, 41, 88, 135]]

In [76]:
response = requests.get(home_url+api_url_extension+"release&id="+anime_id+"&sort=episode_asc")
episodes = json.loads(response.content.decode("UTF-8"))["data"]
test_episode = home_url+"play/"+anime_id+"/"+str(episodes[0]["session"])
test_episode

'https://animepahe.ru/play/b781c27a-dc49-1373-64a9-f40de60ca0ee/ddb34969e076bd8072ed5961e9179a463b677e59f0e5056d41e2256c57266ed5'

In [77]:
response = requests.get(test_episode)
page = response.content
soup = BeautifulSoup(page, "html.parser")
episode_links = soup.find_all("a", class_="dropdown-item", target="_blank")

quality = input("What quality do you want to download in Senpwai uWu? 360p, 720p or 1080p> ")

def quality_chooser(quality="720"):
    if quality == "360p" or quality == "360":
        return 0
    elif quality == "720p" or quality =="720":
        return 1
    elif quality == "1080p" or quality =="1080":
        return 2
    else:
        return "error"

while(True):
    if quality_chooser == "error":
        sub_or_dub = input("Pwease enter a valid choice Senpai> ")
    else:
        break


sub_or_dub = input("Sub or dub> ")

while(True):
    if sub_or_dub.lower() == "sub":
        link = [link["href"] for link in episode_links[:3]]
        break
    elif sub_or_dub.lower() == "dub":
        link = [link["href"] for link in episode_links[3:]]
        print(link)
        break
    else:
        sub_or_dub = input("Please enter a valid choice Senpwai> ")





download_page = requests.get(link[quality_chooser(quality)])

print(link[quality_chooser(quality)])

TypeError: list indices must be integers or slices, not str